In [1]:
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import graphviz
import itertools
#from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
# import Kmeans from sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_absolute_error
from typing import List, Union, Any, Tuple, Dict
import time
from permetrics.regression import RegressionMetric
import oapackage
from scipy.stats import wasserstein_distance, binned_statistic

import warnings
warnings.filterwarnings('ignore')

# Project path
ppath = sys.path[0] + '/../'
sys.path.append(os.path.join(ppath, 'code'))
from discretizers import *

In [2]:
# read json file
import json
with open(os.path.join(ppath, 'scratch', 'glu_bmi_age.pareto.20240725.json'), 'r') as f:
    data = json.load(f)
# cast as dataframe
df = pd.DataFrame(data)
df.head()

,bins,model_accuracy,Glucose.partitioned,BMI.partitioned,Age.partitioned,gpt,impute_accuracy,pareto
0,"[[0, 140, 200], [0, 18.5, 25, 30, 68], [0, 19,...",0.697368,"[140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140...","[68.0, 68.0, 25.0, 68.0, 68.0, 30.0, 68.0, 25....","[65.0, 45.0, 45.0, 45.0, 45.0, 45.0, 45.0, 45....",1.000000,0.908854,1
1,"[[0, 140, 200], [0, 18.5, 25, 30, 68], [0, 2, ...",0.736842,"[140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140...","[68.0, 68.0, 25.0, 68.0, 68.0, 30.0, 68.0, 25....","[50.0, 32.0, 42.0, 22.0, 34.0, 30.0, 28.0, 30....",0.666667,0.834635,0
2,"[[0, 140, 200], [0, 18.5, 25, 30, 68], [0, 5, ...",0.703947,"[140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140...","[68.0, 68.0, 25.0, 68.0, 68.0, 30.0, 68.0, 25....","[50.0, 35.0, 45.0, 25.0, 35.0, 30.0, 30.0, 30....",0.933333,0.842014,0
3,"[[0, 140, 200], [0, 18.5, 25, 30, 68], [0, 10,...",0.703947,"[140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140...","[68.0, 68.0, 25.0, 68.0, 68.0, 30.0, 68.0, 25....","[50.0, 40.0, 50.0, 30.0, 40.0, 30.0, 30.0, 40....",0.933333,0.867622,0
4,"[[0, 140, 200], [0, 18.5, 25, 30, 68], [0, 20,...",0.703947,"[140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140...","[68.0, 68.0, 25.0, 68.0, 68.0, 30.0, 68.0, 25....","[60.0, 40.0, 60.0, 40.0, 40.0, 40.0, 40.0, 40....",0.966667,0.894097,0


In [4]:
# concatenate 'Glucose' 'BMI' and 'Age columns by row 
X = np.array(df[['Glucose.partitioned', 'BMI.partitioned', 'Age.partitioned']])
X

array([[list([140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140.0, 200.0, 140.0, 200.0, 140.0, 200.0, 140.0, 200.0, 140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 200.0, 200.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140.0, 140.0, 200.0, 200.0, 140.0, 200.0, 140.0, 200.0, 140.0, 200.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 200.0, 200.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 200.0, 200.0, 140.0, 140.0, 140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 200.0, 200.0, 140.0, 140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 140.0, 200.0, 140.0, 140.0, 140.0, 140.0, 200.0, 140.0,